# Image Caption Generator (Deep Learning)
**Student Name:** Mazen Mohamed
**Project:** LSTM + VGG16 Image Captioning

**Description:**
This notebook trains a deep learning model to generate captions for images.
1. **Features:** Extracted using VGG16 (Pre-trained).
2. **Architecture:** Merged model (CNN Encoder + LSTM Decoder).
3. **Training:** Trained on Google Colab GPU for 20 Epochs.

In [ ]:
from google.colab import files
print("Please select: features.pkl, tokenizer.pkl, and captions.txt")
uploaded = files.upload()

Please select: features.pkl, tokenizer.pkl, and captions.txt


Saving captions.txt to captions.txt


In [ ]:
!ls

captions.txt  features.pkl  sample_data  tokenizer.pkl


## 1. Environment & Data Setup
Importing necessary libraries and fixing compatibility issues for Google Colab.
Loading the pre-extracted image features and text captions.

## 2. Model Architecture
We use a **Merge Architecture** to combine vision and language:
* **Image Feature Extractor:** VGG16 (Pre-trained) processes the image.
* **Sequence Processor:** LSTM handles the text input.
* **Decoder:** The two inputs are merged (concatenated) and passed through Dense layers to predict the next word.

## 3. Training Execution
* **Epochs:** 20
* **Optimizer:** Adam
* **Loss Function:** Categorical Crossentropy
* **Optimization:** Used `padding='post'` to enable GPU acceleration (CuDNN).

In [ ]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint

# --- 1. SETUP DATA ---
print("Loading files...")

# Load Features
with open('features.pkl', 'rb') as f:
    features = pickle.load(f)

# Load Captions
with open('captions.txt', 'r') as f:
    next(f)
    captions_doc = f.read()

# Create Mapping
mapping = {}
for line in captions_doc.split('\n'):
    tokens = line.split(',')
    if len(tokens) < 2: continue
    image_id, caption = tokens[0], tokens[1:]
    image_id = image_id.split('.')[0]
    caption = " ".join(caption)
    if image_id not in mapping: mapping[image_id] = []
    mapping[image_id].append(caption)

# Clean Text
def clean(mapping):
    import string
    table = str.maketrans('', '', string.punctuation)
    for key, captions in mapping.items():
        for i in range(len(captions)):
            desc = captions[i]
            desc = desc.split()
            desc = [word.lower() for word in desc]
            desc = [word.translate(table) for word in desc]
            desc = [word for word in desc if len(word) > 1]
            desc = [word for word in desc if word.isalpha()]
            desc = 'startseq ' + ' '.join(desc) + ' endseq'
            captions[i] = desc
clean(mapping)

# Rebuild Tokenizer
all_captions = []
for key in mapping:
    for caption in mapping[key]:
        all_captions.append(caption)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)
vocab_size = len(tokenizer.word_index) + 1
max_length = 35
print(f"✅ Setup Complete! Vocab: {vocab_size}, Images: {len(features)}")

# --- 2. DEFINE MODEL ARCHITECTURE ---
inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

inputs2 = Input(shape=(max_length,))
# Embedding Layer (Optimized for GPU stability)
se1 = Embedding(vocab_size, 256)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# --- 3. DATA GENERATOR ---
def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    X1, X2, y = list(), list(), list()
    n = 0
    while 1:
        for key in data_keys:
            n += 1
            if key not in features: continue
            feature = features[key][0]
            captions = mapping[key]
            for caption in captions:
                seq = tokenizer.texts_to_sequences([caption])[0]
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    X1.append(feature)
                    X2.append(in_seq)
                    y.append(out_seq)
            if n == batch_size:
                yield ((np.array(X1), np.array(X2)), np.array(y))
                X1, X2, y = list(), list(), list()
                n = 0

# --- 4. TRAIN ---
print("\n🚀 STARTING TRAINING (Final)...")
epochs = 20
batch_size = 32
steps = len(mapping) // batch_size

checkpoint = ModelCheckpoint('best_model.h5', monitor='loss', save_best_only=True, mode='min')

model.fit(
    data_generator(list(mapping.keys()), mapping, features, tokenizer, max_length, vocab_size, batch_size),
    epochs=epochs,
    steps_per_epoch=steps,
    callbacks=[checkpoint],
    verbose=1
)
print("🎉 DONE! Download 'best_model.h5' and 'tokenizer_colab.pkl'.")

with open('tokenizer_colab.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

Loading files...
✅ Setup Complete! Vocab: 8766, Images: 8091

🚀 STARTING TRAINING (Final)...
Epoch 1/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step - loss: 5.7272

252/252 ━━━━━━━━━━━━━━━━━━━━ 67s 243ms/step - loss: 5.7246
Epoch 2/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step - loss: 4.0248

252/252 ━━━━━━━━━━━━━━━━━━━━ 60s 240ms/step - loss: 4.0243
Epoch 3/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step - loss: 3.5742

252/252 ━━━━━━━━━━━━━━━━━━━━ 60s 238ms/step - loss: 3.5740
Epoch 4/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - loss: 3.3082

252/252 ━━━━━━━━━━━━━━━━━━━━ 61s 241ms/step - loss: 3.3080
Epoch 5/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - loss: 3.1181

...

252/252 ━━━━━━━━━━━━━━━━━━━━ 61s 241ms/step - loss: 2.3552
Epoch 17/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - loss: 2.3242

252/252 ━━━━━━━━━━━━━━━━━━━━ 61s 241ms/step - loss: 2.3241
Epoch 18/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - loss: 2.2931

252/252 ━━━━━━━━━━━━━━━━━━━━ 60s 240ms/step - loss: 2.2930
Epoch 19/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step - loss: 2.2636

252/252 ━━━━━━━━━━━━━━━━━━━━ 60s 240ms/step - loss: 2.2636
Epoch 20/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - loss: 2.2395

252/252 ━━━━━━━━━━━━━━━━━━━━ 61s 241ms/step - loss: 2.2395
🎉 DONE! Download 'best_model.h5' and 'tokenizer_colab.pkl'.


## 4. Model Evaluation (Inference)
Testing the model on a random image from the dataset to verify it generates a coherent sentence.

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import random

# 1. Load the Trained Model
print("Loading model...")
model = load_model('best_model.h5')

# 2. Helper Functions
def idx_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def predict_caption(model, image, tokenizer, max_length=35):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]

        # --- LOGIC FIX: padding='post' to match training ---
        sequence = pad_sequences([sequence], maxlen=max_length, padding='post')

        yhat = model.predict([image, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = idx_to_word(yhat, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text.replace('startseq', '').replace('endseq', '')

# 3. Generate Caption for a Random Image
image_id = random.choice(list(features.keys()))
feature = features[image_id] # Use the pre-loaded feature

print(f"Generating caption for Image ID: {image_id}")
caption = predict_caption(model, feature, tokenizer)

print("\n------------------------------------------------")
print("🤖 AI SAYS:", caption)
print("------------------------------------------------")

## 5. Export Resources
Saving the `tokenizer.json` file to ensure compatibility when running the demo on a local machine (Laptop).

In [ ]:
# 5. EXPORT TOKENIZER (JSON)
import json

tokenizer_json = tokenizer.to_json()

with open('tokenizer.json', 'w') as f:
    f.write(tokenizer_json)

print("🎉 Saved 'tokenizer.json'. Please download this file!")

🎉 Saved 'tokenizer.json'. Please download this file!
